# Simple Mean Reversion

Strategy:

1.If return in the past 5 days > 0.01, we sell

2.If return in the past 5 days < -0.01, we buy

3.We close the position every 10 days 




In [70]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

# Create a Stratey
class MeanReversionStrategy(bt.Strategy):
    params = (
        ('period', 128),
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        self.days_in_trade = 0
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)

        
    def sreturn(self, n):
        """calculate return in the past n days """        
        r = (self.dataclose[0] - self.dataclose[-n])/self.dataclose[-n]
        print('price = ', self.dataclose[-n])
        return r   
    

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        
        r = self.sreturn(5)
       
        # Check if we are in the market
        if not self.position:
            # if we do not have a position
            
            # we MIGHT BUY if return in the past 10 days is less than -1%
            if r < -0.01:
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
                
                self.days_in_trade = 1
            
            # we SELL if return in the past 10 days is more than 1%
            elif r > 0.01:                
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell() 
                
                self.days_in_trade = 1
            
            

        else:
            # if we have a position
            # close the position after holding it for 10 days.            
            if self.dataopen[-9] == self.position.price:
                if self.position.size > 0:
                    # SELL to close position
                    self.log('SELL EMPTY, %.2f' % self.dataclose[0])

                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.sell()
                
                else:
                    
                    # BUY to close position 
                    self.log('BUY EMPTY, %.2f' % self.dataclose[0])

                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.buy()
                
       


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(MeanReversionStrategy)

    datapath = os.path.join('../../../datas/spx-2013-2018.txt')

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        # fromdate=datetime.datetime(2013, 8, 13),
        # Do not pass values before this date
        # todate=datetime.datetime(2018, 2, 9),
        # Do not pass values after this date
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    

    # Write output
    cerebro.addwriter(bt.WriterFile, out='mean_reversion.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00


IndexError: list index out of range